In [1]:
import os, sys
sys.path.append(os.getcwd())

import time
import functools

import numpy as np
import tensorflow as tf
import sklearn.datasets

import tflib as lib
import tflib.ops.linear
import tflib.ops.conv2d
import tflib.ops.batchnorm
import tflib.ops.deconv2d
import tflib.save_images
#import tflib.small_imagenet
import tflib.web_oneclass
import tflib.ops.layernorm
import tflib.plot
import os

DIM = 32 # Model dimensionality
DEVICES = 0
NUM_SAMPLE = 100
TARGET_SIZE = 128
OUTPUT_DIM = TARGET_SIZE*TARGET_SIZE*3 # Number of pixels in each iamge


In [2]:
def LeakyReLU(x, alpha=0.2):
    return tf.maximum(alpha*x, x)

def ReLULayer(name, n_in, n_out, inputs):
    output = lib.ops.linear.Linear(name+'.Linear', n_in, n_out, inputs, initialization='he')
    return tf.nn.relu(output)

def LeakyReLULayer(name, n_in, n_out, inputs):
    output = lib.ops.linear.Linear(name+'.Linear', n_in, n_out, inputs, initialization='he')
    return LeakyReLU(output)

def Batchnorm(name, axes, inputs):
    if ('Discriminator' in name) and (MODE == 'wgan-gp'):
        if axes != [0,2,3]:
            raise Exception('Layernorm over non-standard axes is unsupported')
        return lib.ops.layernorm.Layernorm(name,[1,2,3],inputs)
    else:
        return lib.ops.batchnorm.Batchnorm(name,axes,inputs,fused=True)

def DCGANGenerator_128(n_samples, noise=None, dim=DIM, bn=True, nonlinearity=tf.nn.relu):
    lib.ops.conv2d.set_weights_stdev(0.02)
    lib.ops.deconv2d.set_weights_stdev(0.02)
    lib.ops.linear.set_weights_stdev(0.02)

    if noise is None:
        noise = tf.random_normal([n_samples, 128])

    output = lib.ops.linear.Linear('Generator.Input', 128, 4*4*16*dim, noise)
    output = tf.reshape(output, [-1, 16*dim, 4, 4])
    if bn:
        output = Batchnorm('Generator.BN1', [0,2,3], output)
    output = nonlinearity(output)

    output = lib.ops.deconv2d.Deconv2D('Generator.2', 16*dim, 8*dim, 5, output)
    if bn:
        output = Batchnorm('Generator.BN2', [0,2,3], output)
    output = nonlinearity(output)

    output = lib.ops.deconv2d.Deconv2D('Generator.3', 8*dim, 4*dim, 5, output)
    if bn:
        output = Batchnorm('Generator.BN3', [0,2,3], output)
    output = nonlinearity(output)

    output = lib.ops.deconv2d.Deconv2D('Generator.4', 4*dim, 2*dim, 5, output)
    if bn:
        output = Batchnorm('Generator.BN4', [0,2,3], output)
    output = nonlinearity(output)

    output = lib.ops.deconv2d.Deconv2D('Generator.5', 2*dim, dim, 5, output)
    if bn:
        output = Batchnorm('Generator.BN5', [0,2,3], output)
    output = nonlinearity(output)

    output = lib.ops.deconv2d.Deconv2D('Generator.6', dim, 3, 5, output)
    output = tf.tanh(output)

    lib.ops.conv2d.unset_weights_stdev()
    lib.ops.deconv2d.unset_weights_stdev()
    lib.ops.linear.unset_weights_stdev()

    return tf.reshape(output, [-1, OUTPUT_DIM])

def nchw_to_nhwc(x):
    return tf.transpose(x, [0,2,3,1])
def nhwc_to_nchw(x):
    return tf.transpose(x, [0,3,1,2])
def BEGANGenerator(n_samples, noise=None, dim=DIM):
    if noise is None:
        noise = tf.random_normal([n_samples, 128])

    output = lib.ops.linear.Linear('Generator.Input', 128, 8*8*dim, noise)
    output = tf.reshape(output, [-1, dim, 8, 8])

    repeat_num = int(np.log2(TARGET_SIZE/8)) + 1
    curr_shape = 8
    for idx in range(repeat_num):
        output = lib.ops.conv2d.Conv2D('Generator.{}a'.format(idx+2), dim, dim, 3, output, stride=1)
        output = LeakyReLU(output)
        output = lib.ops.conv2d.Conv2D('Generator.{}b'.format(idx+2), dim, dim, 3, output, stride=1)
        output = LeakyReLU(output)
        if idx < repeat_num - 1:
            output = nchw_to_nhwc(output)
            output = tf.image.resize_nearest_neighbor(output, (curr_shape*2, curr_shape*2))
            output = nhwc_to_nchw(output)
            curr_shape = curr_shape * 2
    print(curr_shape)

    output = lib.ops.conv2d.Conv2D('Generator.{}'.format(repeat_num+2), dim, 3, 3, output)
    output = tf.tanh(output)

    return tf.reshape(output, [-1, OUTPUT_DIM])

In [5]:
with tf.Session() as session:
    Generator = BEGANGenerator
    # For generating samples
    fixed_noise = tf.constant(np.random.normal(size=(NUM_SAMPLE, 128)).astype('float32'))
    all_fixed_noise_samples = Generator(NUM_SAMPLE, noise=fixed_noise)
    if tf.__version__.startswith('1.'):
        all_fixed_noise_samples = tf.concat(all_fixed_noise_samples, axis=0)
    else:
        all_fixed_noise_samples = tf.concat(0, all_fixed_noise_samples)
    saver = tf.train.Saver()
    saver.restore(session, 'models/BEGAN128_wgan-gp_horse_Mhidden32/model.ckpt-100000')
    samples = session.run(all_fixed_noise_samples)
    samples = ((samples+1.)*(255.99/2)).astype('int32')
    lib.save_images.save_images(samples.reshape((NUM_SAMPLE, 3, TARGET_SIZE, TARGET_SIZE)), 'tmp.png')

128
